In [2]:
import torch.nn as nn
from torch.nn import functional as F
import pandas as pd
import matplotlib.pyplot as plt
import os
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import torch
import numpy as np
import glob
import segmentation_models_pytorch as smp
from sklearn.model_selection import train_test_split
import time
from torch.autograd import Variable

In [3]:
import random
import tqdm
import cv2

## Метрики с которыми надо поэкспериментировать

In [ ]:
#https://www.kaggle.com/lyakaap/weighing-boundary-pixels-loss-script-by-keras2

# weight: weighted tensor(same shape with mask image)
def weighted_bce_loss(y_true, y_pred, weight):
    # avoiding overflow
    epsilon = 1e-7
    y_pred = K.clip(y_pred, epsilon, 1. - epsilon)
    logit_y_pred = K.log(y_pred / (1. - y_pred))
    
    # https://www.tensorflow.org/api_docs/python/tf/nn/weighted_cross_entropy_with_logits
    loss = (1. - y_true) * logit_y_pred + (1. + (weight - 1.) * y_true) * \
    (K.log(1. + K.exp(-K.abs(logit_y_pred))) + K.maximum(-logit_y_pred, 0.))
    return K.sum(loss) / K.sum(weight)

def weighted_dice_loss(y_true, y_pred, weight):
    smooth = 1.
    w, m1, m2 = weight * weight, y_true, y_pred
    intersection = (m1 * m2)
    score = (2. * K.sum(w * intersection) + smooth) / (K.sum(w * m1) + K.sum(w * m2) + smooth)
    loss = 1. - K.sum(score)
    return loss

def weighted_bce_dice_loss(y_true, y_pred):
    y_true = K.cast(y_true, 'float32')
    y_pred = K.cast(y_pred, 'float32')
    # if we want to get same size of output, kernel size must be odd number
    averaged_mask = K.pool2d(
            y_true, pool_size=(11, 11), strides=(1, 1), padding='same', pool_mode='avg')
    border = K.cast(K.greater(averaged_mask, 0.005), 'float32') * K.cast(K.less(averaged_mask, 0.995), 'float32')
    weight = K.ones_like(averaged_mask)
    w0 = K.sum(weight)
    weight += border * 2
    w1 = K.sum(weight)
    weight *= (w0 / w1)
    loss = weighted_bce_loss(y_true, y_pred, weight) + \
    weighted_dice_loss(y_true, y_pred, weight)
    return loss

## Разбор дообучения моделей и заморозки слоев

In [48]:
model = smp.Unet('mobilenet_v2', classes=1, encoder_depth=5, 
                 encoder_weights='imagenet', decoder_channels = [256, 128, 64, 32, 16])

In [7]:
#model

In [32]:
for (name, module) in model.named_modules():
    print(name)


encoder
encoder.features
encoder.features.0
encoder.features.0.0
encoder.features.0.1
encoder.features.0.2
encoder.features.1
encoder.features.1.conv
encoder.features.1.conv.0
encoder.features.1.conv.0.0
encoder.features.1.conv.0.1
encoder.features.1.conv.0.2
encoder.features.1.conv.1
encoder.features.1.conv.2
encoder.features.2
encoder.features.2.conv
encoder.features.2.conv.0
encoder.features.2.conv.0.0
encoder.features.2.conv.0.1
encoder.features.2.conv.0.2
encoder.features.2.conv.1
encoder.features.2.conv.1.0
encoder.features.2.conv.1.1
encoder.features.2.conv.1.2
encoder.features.2.conv.2
encoder.features.2.conv.3
encoder.features.3
encoder.features.3.conv
encoder.features.3.conv.0
encoder.features.3.conv.0.0
encoder.features.3.conv.0.1
encoder.features.3.conv.0.2
encoder.features.3.conv.1
encoder.features.3.conv.1.0
encoder.features.3.conv.1.1
encoder.features.3.conv.1.2
encoder.features.3.conv.2
encoder.features.3.conv.3
encoder.features.4
encoder.features.4.conv
encoder.featur

In [33]:
for (name, module) in model.named_children():
    if name == 'encoder':
        for layer in module.children():
            print(layer)

Sequential(
  (0): ConvBNActivation(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU6(inplace=True)
  )
  (1): InvertedResidual(
    (conv): Sequential(
      (0): ConvBNActivation(
        (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU6(inplace=True)
      )
      (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (2): InvertedResidual(
    (conv): Sequential(
      (0): ConvBNActivation(
        (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU6(in

In [38]:
layer_counter = 0
for (name, module) in model.named_children():
    if name == 'encoder':
        for layer in module.children():
            for param in layer.parameters():
                param.requires_grad = False
            
            print('Layer "{}" in module "{}" was frozen!'.format(layer_counter, name))
            layer_counter+=1

Layer "0" in module "encoder" was frozen!


In [ ]:
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, vgg16.parameters()), lr=0.001)

In [ ]:
 for k,v in model.named_parameters():
     if k.startswith('conv1') or k.startswith('layer1'):
         v.requires_grad = False

In [41]:
from torchvision import models
from torchsummary import summary

In [45]:
summary(model, (3, 512, 512))

RuntimeError: Input type (torch.cuda.FloatTensor) and weight type (torch.FloatTensor) should be the same

## Сравнение скорости работы OpenCV и PIL

In [102]:
src_dir = '/home/dima/carvana_dataset/train/train'
dst_dir = '/home/dima/carvana_dataset/train/train_small'

In [110]:
def pil_get_image_rw_time(src_dir: str, dst_dir: str, max_img: int=100):
    
    train_files = os.listdir(src_dir)
    random.shuffle(train_files)

    for i, f in tqdm.tqdm(enumerate(train_files)):
        img_pil = Image.open(os.path.join(src_dir, f))
        img_pil = np.array(img_pil)
        img_pil = Image.fromarray(np.uint8(img_pil)).convert('RGB')
        img_pil.save(os.path.join(dst_dir, f))
        #print(type(img_pil))
        if i >= max_img:
            break

In [111]:
def cv_2_get_image_rw_time(src_dir: str, dst_dir: str, max_img: int=100):
    
    train_files = os.listdir(src_dir)
    random.shuffle(train_files)

    for i, f in tqdm.tqdm(enumerate(train_files)):
        img_cv2 = cv2.imread(os.path.join(src_dir, f))
        cv2.imwrite(os.path.join(dst_dir, f), img_cv2)
        if i >= max_img:
            break

In [112]:
!rm /home/dima/carvana_dataset/train/train_small/*
time1 = time.time()
pil_get_image_rw_time(src_dir, dst_dir, max_img=100)
time2 = time.time()
print(time2-time1)

100it [00:05, 17.11it/s]

5.86529278755188


In [113]:
!rm /home/dima/carvana_dataset/train/train_small/*
time1 = time.time()
cv_2_get_image_rw_time(src_dir, dst_dir, max_img=100)
time2 = time.time()
print(time2-time1)

100it [00:03, 25.98it/s]

3.8542134761810303


## Определение среднего и дисперсии набора изображений по каналам

In [7]:
src_dir = '/home/dima/carvana_dataset/train/train'

In [8]:
def calculate_mean_image(src_dir: str, max_img: int=100):
    img_mean_sum = np.zeros(3, np.float64)
    img_var_sum = np.zeros(3, np.float64)

    train_files = os.listdir(src_dir)
    random.shuffle(train_files)

    for i, f in tqdm.tqdm(enumerate(train_files)):
        image = cv2.imread(os.path.join(src_dir, f))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype('float')/255.0
        channels = image.reshape((-1, 3))
        img_mean_sum += channels.mean(axis=0)
        img_var_sum += channels.var(axis=0)
        if i >= max_img:
            break

    chan_mean = img_mean_sum / (i + 1)
    chan_std = (img_var_sum / (i + 1)) ** 0.5

    print('channels mean:', chan_mean)
    print('channels std:', chan_std)
    #np.save(CHANNELS_MEAN, chan_mean)
    #np.save(CHANNELS_STD, chan_std)
    return chan_mean, chan_std

In [9]:
chan_mean, chan_std = calculate_mean_image(src_dir, max_img=100)

100it [00:16,  5.97it/s]

channels mean: [0.69606284 0.68972409 0.68443157]
channels std: [0.23977361 0.24345116 0.24009687]


In [5]:
# Среднее и дисперсия для imagenet
from segmentation_models_pytorch.encoders import get_preprocessing_fn
preprocess_input = get_preprocessing_fn('mobilenet_v2', pretrained='imagenet')
preprocess_input

functools.partial(<function preprocess_input at 0x7fcbc0b0c550>, input_space='RGB', input_range=[0, 1], mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

## Преобразование масок из gif в jpeg

In [37]:
import os
import tqdm
from PIL import Image
import numpy as np
import cv2
from torch.utils.data import Dataset, DataLoader
import glob
import pandas as pd

In [38]:
src_dir = '/home/dima/carvana_dataset/train_masks/train_masks'
dst_dir = '/home/dima/carvana_dataset/train_masks/train_masks_jpg'

def gif_to_jpg(src_dir: str, src_dst: str):
    train_files = os.listdir(src_dir)
    for i, f in tqdm.tqdm(enumerate(train_files)):
        mask = Image.open(os.path.join(src_dir, f))  
        mask.save(os.path.join(dst_dir, f).split('.')[0] + '.png')

In [39]:
gif_to_jpg(src_dir, src_dst)

5088it [00:46, 108.42it/s]


In [43]:
image = cv2.imread('/home/dima/carvana_dataset/train_masks/train_masks_png/4fc2470c0f7b_13_mask.png')

In [44]:
np.unique(image)

array([  0, 255], dtype=uint8)

In [45]:
image.shape

(1280, 1918, 3)

## Работа с ONNX

In [52]:
import onnx
import onnxruntime
import torch
import numpy as np
import albumentations as A
import cv2
from albumentations.pytorch import ToTensorV2

In [3]:
#sudo apt install nvidia-cuda-toolkit

In [1]:
!nvidia-smi

Thu Oct  7 16:13:17 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.63.01    Driver Version: 470.63.01    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:01:00.0  On |                  N/A |
| 37%   62C    P2   104W / 170W |  11450MiB / 12045MiB |     41%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
torch.cuda.get_device_name(torch.cuda.current_device()) 

'NVIDIA GeForce RTX 3060'

In [6]:
torch.cuda.is_available() 

True

In [7]:
onnxruntime.get_device()

'GPU'

In [8]:
torch.backends.cudnn.is_available()

True

In [9]:
torch.backends.cudnn.version()

8005

In [10]:
torch.backends.cuda.is_built()

True

In [18]:
class ONNX_model(object):
    def __init__(self, model_addr: str):
        super(ONNX_model, self).__init__()
        self.model_addr = model_addr
    
    def run_session(self):
        ort_session = onnxruntime.InferenceSession(self.model_addr)
        input_name = ort_session.get_inputs()[0].name
    
    def check(self):
        onnx_model = onnx.load(self.model_addr)
        print(onnx.checker.check_model(onnx_model))
    
    def predict(self):
        ort_inputs = {input_name: np.random.randn(2, 3, 512, 512).astype(np.float32)}
        return ort_session.run(None, ort_inputs)

a = ONNX_model('./carvana_model.onnx')

In [74]:
ort_session = onnxruntime.InferenceSession('./carvana_model.onnx')
input_name = ort_session.get_inputs()[0].name
ort_inputs = {input_name: np.random.randn(2, 3, 1024, 1024).astype(np.float32)}
ort_outs = ort_session.run(None, ort_inputs)

In [76]:
ort_outs[0].shape

(2, 1, 1024, 1024)

In [21]:
class CustomDatasetForONNX(Dataset):
    '''Класс для создания тестовых датасетов'''
    def __init__(self, data_info, transform: object):
        '''Входные параметры:
        data_info: pd.DataFrame - датафрейм с адресами и именами изображений
        transform: object - список трансформации, которым будут подвергнуты изображения
        Возвращаемые значения:
        объект класса CustomDatasetForTest'''
        # Подаем наш подготовленный датафрейм
        self.data_info = data_info
        # Получаем адреса RGB изображений 
        self.image_addresses = self.data_info.iloc[:,0]
        # Получаем имена RGB изображений 
        self.image_names = self.data_info.iloc[:,1]
        # Количество пар картинка-сегментация
        self.data_len = len(self.data_info.index)
        # Сохраняем преобразования данных
        self.transform = transform

    def __getitem__(self, index):
        '''Входные параметры:
        img: int - индекс для обращения к элементам датафрейма data_info
        Возвращаемые значения:
        img: torch.Tensor - тензорное представление изображения с размерностью out_shape
        mask_small: torch.Tensor - тензорное представление маски с исходной размерностью
        image_name: str - имя изображения'''
        image = cv2.imread(self.image_addresses[index])
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype('float')/255.0
        
        transformed = self.transform(image=image)
        tr_image = transformed['image']
        image_name = self.image_names[index]
    
        return (index, tr_image, image_name)

    def __len__(self):
        return self.data_len

In [22]:
predict_directory = '/home/dima/carvana_dataset/test/predict_small/'
test_dataset = '/home/dima/carvana_dataset/test/test/'

In [27]:
test_dataframe = {}
test_dataframe['img_addr'] = list(glob.glob(test_dataset + "/*"))
test_dataframe = pd.DataFrame(test_dataframe)
test_dataframe['img_name'] = test_dataframe['img_addr'].apply(lambda x: x.split('/')[-1])

In [53]:
valid_transform = A.Compose([
    A.Resize(1024, 1024, cv2.INTER_LINEAR),
    ToTensorV2(),
])

In [54]:
test_data = CustomDatasetForONNX(test_dataframe, valid_transform)
test_data_loader = DataLoader(test_data, batch_size=2, shuffle=False)

In [55]:
aa = iter(test_data_loader)

In [56]:
a = aa.next()

In [58]:
a[0].shape

torch.Size([2])

In [63]:
b = a[1].numpy()

In [65]:
b.shape

(2, 3, 1024, 1024)

In [69]:
def sigmoid(x):
    return 1/(1 + np.exp(-x))

In [85]:
import time
from torch.nn import functional as F

In [91]:
def tensor_to_rle(tensor: torch.Tensor) -> str:
    with torch.no_grad():
        tensor = tensor.view(1, -1)
        tensor = tensor.squeeze(0)
        tensor[0] = 0
        tensor[-1] = 0
        rle = torch.where(tensor[1:] != tensor[:-1])[0] + 2
        rle[1::2] = rle[1::2] - rle[:-1:2]
        rle = rle.cpu().detach().numpy()
        rle_str = rle_to_string(rle)
        return rle_str
    
def rle_to_string(runs: torch.Tensor) -> str:
    return ' '.join(str(x) for x in runs)

In [92]:
img_names = []
img_rles = []
time1 = time.time()
time2 = time.time()
mask_treashold = 0.5

ort_session = onnxruntime.InferenceSession('./carvana_model.onnx')
input_name = ort_session.get_inputs()[0].name

for batch_idx, (index, img, img_name)  in enumerate(test_data_loader):

    img = img.numpy()
    ort_inputs = {input_name: img.astype(np.float32)}
    ort_outs = ort_session.run(None, ort_inputs)
    pred_mask_logit = torch.from_numpy(ort_outs[0])
    
    pred_mask_logit = F.interpolate(input=pred_mask_logit, size=(1280, 1918), mode='nearest')
    pred_mask_logit_prob = torch.sigmoid(pred_mask_logit)
    pred_mask = torch.where(pred_mask_logit_prob > mask_treashold, 1, 0)
            
    # Каждое изображение в тензоре преобразуем в картинку и сохраняем
    for i in range(pred_mask.shape[0]):
        #if predict_directory != None:
        #    mask = (pred_mask[i].cpu().numpy() * 255.0)[0] # [0] - избавляемся от батч размерности
        #    PIL_image = Image.fromarray(mask.astype('uint8'), 'L')
        #    PIL_image.save((predict_directory+img_name[i]).split('.')[0]+'.gif')
                
        # Если требуется, получаем значения rle для каждой картинки

        img_names.append(img_name[i])
        img_rles.append(tensor_to_rle(pred_mask[i]))
            
    if (batch_idx+1) % 300 == 0:
            print('-'*50)
            print(f'Processed images: {(batch_idx+1)*img.shape[0]}')
            time3 = time.time()
            print(f'Total time: {(time3-time1):.2f} sec')
            print(f'Time to process {300*img.shape[0]} images: {(time3-time2):.2f} sec')
            time2 = time.time()
                

rle_dataframe = pd.DataFrame(list(zip(img_names, img_rles)), columns =['img_name', 'img_rle'])

--------------------------------------------------
Processed images: 600
Total time: 299.56 sec
Time to process 600 images: 299.56 sec


KeyboardInterrupt: 